In [5]:
import pymc as pm
import numpy as np
import pandas as pd


data = pd.read_csv('Ice Cream Sales - temperatures.csv')

X = data.drop(data.columns[-1], axis=1).values
y = data[data.columns[-1]].values
p = X.shape[1]

with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros(p), cov=np.eye(p), shape=p)
    
    sigma = pm.HalfNormal('sigma', sigma=1)
    
    y_observed = pm.Normal('y_observed', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [betas, sigma]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 2 seconds.


In [8]:
import pymc as pm
import numpy as np
import pandas as pd

data = pd.read_csv('Ice Cream Sales - temperatures.csv')


y = data.iloc[:, -1].values
X = data.iloc[:, :-1].values
p = X.shape[1]  

with pm.Model() as MNV_LKJ:
    
    packed_L = pm.LKJCholeskyCov('packed_L', n=p, eta=2.0,
                                  sd_dist=pm.Exponential.dist(1.0, shape=p), compute_corr=False)
    L = pm.expand_packed_triangular(p, packed_L)
    Sigma = pm.Deterministic('Sigma', L.dot(L.T))  
    
    mu = pm.MvNormal('mu', mu=np.zeros(p), cov=np.eye(p)*1e-6, shape=p)

    y_obs = pm.MvNormal('y_obs', mu=mu, chol=L, observed=y)

    trace = pm.sample(1600)


IndexError: index 1 is out of bounds for axis 2 with size 1
Apply node that caused the error: _lkjcholeskycovbase_rv{1, (0, 0, 1), floatX, True}(RandomGeneratorSharedVariable(<Generator(PCG64) at 0x7FC565047760>), [], 11, 1, 2.0, exponential_rv{0, (0,), floatX, True}.out)
Toposort index: 1
Inputs types: [RandomGeneratorType, TensorType(int64, shape=(0,)), TensorType(int64, shape=()), TensorType(int32, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=(1,))]
Inputs shapes: ['No shapes', (0,), (), (), (), (1,)]
Inputs strides: ['No strides', (0,), (), (), (), (8,)]
Inputs values: [Generator(PCG64) at 0x7FC565047760, array([], dtype=int64), array(11), array(1, dtype=int32), array(2.), array([3.29352779])]
Outputs clients: [['output'], [Second(packed_L, [0.])]]

HINT: Re-running with most PyTensor optimizations disabled could provide a back-trace showing when this node was created. This can be done by setting the PyTensor flag 'optimizer=fast_compile'. If that does not wo